In [44]:
import pandas as pd
import numpy as np
dataframe = pd.read_csv('./washington_data_m_n_predict6.csv',index_col=0)

In [ ]:
dataframe['M_pattern'].value_counts()

In [47]:
dataframe['M_pattern'].replace(['none'], 0, inplace=True)
dataframe['M_pattern'].replace(['1_work'], 1, inplace=True)
dataframe['M_pattern'].replace(['2_work'], 2, inplace=True)
dataframe['M_pattern'].replace(['3_work'], 3, inplace=True)
dataframe['M_pattern'].replace(['1_edu'], 4, inplace=True)
dataframe['M_pattern'].replace(['1_work_1_edu'], 5, inplace=True)
dataframe['M_pattern'].replace(['1_sch_1_edu'], 6, inplace=True)
dataframe['M_pattern'].replace(['1_sch'], 7, inplace=True)


In [48]:
def more_than_4(trip_n):
  if trip_n > 4:
     n = 4
  else:
    n = trip_n
  return n

In [49]:
#dataframe = dataframe[dataframe['OTHER'] != 0]

In [50]:
dataframe['OTHER_less_than_4'] = dataframe.apply(lambda row: more_than_4(row['OTHER']), axis=1)

In [ ]:
dataframe['M_pattern'].value_counts()

In [52]:
dataframe.dropna(axis=0, inplace=True)

In [53]:
cols_to_norm = ['hhwrk', 'AVG_VEH_CNT', 'MTLRTSHR', 'MTLRTLNG', 'ALLPKSHR', 'ALLPKLNG', 'ALLOPSHR', 'ALLOPLNG', 'popden', 'empden']

#creating a dataframe with only the features we want to train our models on							
dataframe_train = dataframe[cols_to_norm]
# normalizing data
dataframe_train = dataframe_train.apply(lambda x: (x - x.mean()) / (x.max()-x.min())) # 

In [54]:
income = pd.get_dummies(dataframe.HHFAMINC, prefix='INCOME')
age = pd.get_dummies(dataframe.R_AGE_CAT, prefix='AGE')
#relate = pd.get_dummies(dataframe.relate, prefix='relate')

In [55]:
X = pd.concat([dataframe_train, income, age, dataframe.R_SEX, dataframe.university_student,
               dataframe.Student_Age_5_15, dataframe.Student_Age_16_up] , axis=1)

In [56]:
y = dataframe.M_pattern.values

In [57]:
#splitting Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

In [58]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from keras.models import Sequential

model = Sequential()
model.add(Dense(200, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(8, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])

In [ ]:
model.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
          batch_size=32,epochs=100)

In [ ]:
loss_df = pd.DataFrame(model.history.history)
loss_df.plot(figsize=(12,8))

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
#predict mandatory trips for train data
y_pred_X = model.predict(X)

df1 = pd.DataFrame(y_pred_X, columns =['prob1', 'prob2', 'prob3', 'prob4', 'prob5','prob6', 'prob7', 'prob8'])

X.reset_index(drop=True, inplace=True)
#add predictions to the dataset
X.join(df1)

In [71]:
np.argmax(y_pred[0])

1

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

In [ ]:
# Recall
from sklearn.metrics import recall_score
recall_score(y_test, y_pred, average=None)

In [ ]:
# Precision
from sklearn.metrics import precision_score
precision_score(y_test, y_pred, average=None)

In [ ]:
# f1 score
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average=None)

In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(y_test, y_pred)

#non mandatory

In [93]:
dataframe['OTHER'] = dataframe['OTHER'].apply(lambda x: 4 if x>=4 else x)

In [94]:
y = dataframe['OTHER']

In [96]:
#splitting Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

In [97]:
model2 = Sequential()
model2.add(Dense(200, input_shape=(X_train.shape[1],), activation='relu'))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(5, activation='softmax'))

model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])

In [ ]:
model2.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
          batch_size=32,epochs=100)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = model2.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

In [ ]:
# Recall
from sklearn.metrics import recall_score
recall_score(y_test, y_pred, average=None)

In [ ]:
# Precision
from sklearn.metrics import precision_score
precision_score(y_test, y_pred, average=None)

In [ ]:
# f1 score
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average=None)

In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(y_test, y_pred)